In [ ]:
import pandas as pd
df = pd.read_csv("hmeq-1.csv")
df.head()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import statsmodels
import re

In [ ]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score, recall_score, precision_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.impute import KNNImputer

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score

In [ ]:
import pandas.core.algorithms as algos
import scipy.stats.stats as stats
import traceback
import string

In [ ]:
df[['BAD']].value_counts(normalize=True)

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
missing_percent = df.isnull().mean().sort_values(ascending=False) * 100
print(missing_percent)

In [ ]:
categorical_features = [col for col in df.columns if df[col].dtype == 'object']
numerical_features = [col for col in df.columns if df[col].dtype != 'object']

print('Number of categorical features:', len(categorical_features))
print('Number of numerical features:', len(numerical_features))

In [ ]:
# Đếm tần số của từng giá trị trong mỗi feature theo BAD
for feature in categorical_features:
    print(f"\nTần số của {feature} theo BAD:")
    freq_table = pd.crosstab(df[feature], df['BAD'], dropna=False)  # Giữ giá trị NaN nếu có
    print(freq_table)

In [ ]:
df['REASON'] = df['REASON'].fillna('DebtCon')
df

In [ ]:
df.loc[(df['JOB'].isnull()) & (df['BAD'] == 0), 'JOB'] = 'Office'
df.loc[(df['JOB'].isnull()) & (df['BAD'] == 1), 'JOB'] = 'Other'

for feature in categorical_features:
    print(f"\nTần số của {feature} theo BAD:")
    freq_table = pd.crosstab(df[feature], df['BAD'], dropna=False)  # Giữ giá trị NaN nếu có
    print(freq_table)

In [ ]:
median_value_bad_0 = df[df['BAD'] == 0]['VALUE'].median()
median_value_bad_1 = df[df['BAD'] == 1]['VALUE'].median()

# Điền giá trị thiếu theo BAD
df.loc[(df['VALUE'].isna()) & (df['BAD'] == 0), 'VALUE'] = median_value_bad_0
df.loc[(df['VALUE'].isna()) & (df['BAD'] == 1), 'VALUE'] = median_value_bad_1

In [ ]:
cols_median_by_bad = ['CLNO', 'YOJ', 'CLAGE', 'NINQ', 'DELINQ', 'DEROG']
for col in cols_median_by_bad:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    median_bad_0 = df[df['BAD'] == 0][col].median()
    median_bad_1 = df[df['BAD'] == 1][col].median()
    df.loc[(df[col].isna()) & (df['BAD'] == 0), col] = median_bad_0
    df.loc[(df[col].isna()) & (df['BAD'] == 1), col] = median_bad_1

In [ ]:
features_mortdue = ['CLNO', 'VALUE', 'LOAN', 'BAD', 'MORTDUE']
df_mortdue_impute = df[features_mortdue].copy()

# Đảm bảo chỉ số đồng bộ
df_mortdue_impute.index = df.index

# Xử lý giá trị không phải số
for col in features_mortdue:
    df_mortdue_impute[col] = pd.to_numeric(df_mortdue_impute[col], errors='coerce')

# Kiểm tra NaN trước khi chạy KNN
print("\nSố lượng NaN trong các feature trước khi điền MORTDUE:")
print(df_mortdue_impute.isna().sum())

# Áp dụng KNN Imputer
imputer_mortdue = KNNImputer(n_neighbors=5)
df_mortdue_filled = imputer_mortdue.fit_transform(df_mortdue_impute)

# Chuyển đổi ngược về DataFrame
df_mortdue_filled = pd.DataFrame(df_mortdue_filled, columns=features_mortdue, index=df.index)

# Gán lại giá trị MORTDUE
df['MORTDUE'] = df_mortdue_filled['MORTDUE']

# 4. Kiểm tra số lượng giá trị thiếu sau khi điền
print("\nSố lượng giá trị thiếu trong MORTDUE sau khi điền:", df['MORTDUE'].isna().sum())

In [ ]:
# 3. Điền DEBTINC bằng KNN Imputation
features_debtinc = ['CLNO', 'MORTDUE', 'NINQ', 'LOAN', 'VALUE', 'BAD', 'DEBTINC']
df_debtinc_impute = df[features_debtinc].copy()

# Đảm bảo chỉ số đồng bộ
df_debtinc_impute.index = df.index

# Xử lý giá trị không phải số
for col in features_debtinc:
    df_debtinc_impute[col] = pd.to_numeric(df_debtinc_impute[col], errors='coerce')

# Kiểm tra NaN trước khi chạy KNN
print("\nSố lượng NaN trong các feature trước khi điền DEBTINC:")
print(df_debtinc_impute.isna().sum())

# Áp dụng KNN Imputer
imputer_debtinc = KNNImputer(n_neighbors=5)
df_debtinc_filled = imputer_debtinc.fit_transform(df_debtinc_impute)

# Chuyển đổi ngược về DataFrame
df_debtinc_filled = pd.DataFrame(df_debtinc_filled, columns=features_debtinc, index=df.index)

# Gán lại giá trị DEBTINC
df['DEBTINC'] = df_debtinc_filled['DEBTINC']

# 4. Kiểm tra số lượng giá trị thiếu sau khi điền
print("\nSố lượng giá trị thiếu trong DEBTINC sau khi điền:", df['DEBTINC'].isna().sum())

In [ ]:
# Tính ma trận tương quan
# Chọn chỉ các cột số để tính toán ma trận tương quan
correlation_matrix = df[numerical_features].corr()

# Vẽ heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Heatmap of Correlation Matrix')
plt.show()

In [ ]:
# Lọc toàn bộ outlier theo IQR cho các biến số
def remove_outliers_iqr(df_input, columns):
    df_temp = df_input.copy()
    for col in columns:
        Q1 = df_temp[col].quantile(0.25)
        Q3 = df_temp[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_temp = df_temp[(df_temp[col] >= lower_bound) & (df_temp[col] <= upper_bound)]
    return df

# Lấy danh sách các cột số (trừ cột target)
num_cols = df.select_dtypes(include=['float64', 'int64']).columns.drop('BAD')

# Áp dụng lọc outlier
df_clean = remove_outliers_iqr(df, num_cols)

In [ ]:
import pandas as pd
import numpy as np

def calculate_woe_iv(df, feature, target, bins=10):
    # Chia bin nếu là continuous feature
    if pd.api.types.is_numeric_dtype(df[feature]):
        df['bucket'] = pd.qcut(df[feature], q=bins, duplicates='drop')
    else:
        df['bucket'] = df[feature]

    # Tạo bảng tổng hợp
    woe_df = df.groupby('bucket').agg({target: ['sum', 'count']})
    woe_df.columns = ['Bad', 'Total']
    woe_df['Good'] = woe_df['Total'] - woe_df['Bad']

    # Tính tỷ lệ % Good và % Bad
    woe_df['% Good'] = woe_df['Good'] / woe_df['Good'].sum()
    woe_df['% Bad'] = woe_df['Bad'] / woe_df['Bad'].sum()

    # Tính WOE và IV
    woe_df['WOE'] = np.log(woe_df['% Good'] / woe_df['% Bad'])
    woe_df['IV'] = (woe_df['% Good'] - woe_df['% Bad']) * woe_df['WOE']

    # Tổng IV cho toàn bộ feature
    iv = woe_df['IV'].sum()

    return iv, woe_df[['Bad', 'Good', 'Total', '% Good', '% Bad', 'WOE', 'IV']]

# Danh sách các cột cần tính IV
features = ["DEBTINC", "MORTDUE", "VALUE", "DEROG", "CLAGE", "NINQ", "DELINQ", "YOJ", "CLNO", "REASON", "JOB"]
iv_scores = {}

# Tính IV cho từng feature và lưu kết quả
for feature in features:
    # Check if feature exists in the dataframe before calculating IV
    if feature in df.columns:
        iv, detail = calculate_woe_iv(df, feature, "BAD")
        # Only store and print if IV calculation was successful (iv is not 0 and detail is not None)
        if detail is not None:
            iv_scores[feature] = iv
            print(f"Feature: {feature}, IV: {iv:.4f}")
        else:
            print(f"Feature: {feature}, IV cannot be calculated (all target values are the same or missing).")
    else:
        print(f"Feature: {feature} not found in DataFrame.")


# Sắp xếp các feature theo IV từ cao xuống thấp
# Convert the dictionary to a list of tuples for sorting
iv_sorted = sorted(iv_scores.items(), key=lambda x: x[1], reverse=True)

# In kết quả
print("\nSorted IV scores:") # This line is corrected
for feature, iv in iv_sorted:
    print(f"{feature}: {iv:.4f}")

In [ ]:
# Xóa cột 'REASON'
df = df.drop('REASON', axis=1)
df = df.drop('bucket', axis=1)

# Kiểm tra lại các cột sau khi xóa
print("Các cột sau khi xóa 'REASON':")
print(df.columns)

In [ ]:
# Điền missing value trước khi encode
df["JOB"] = df["JOB"].fillna("Unknown")

# Thực hiện One-Hot Encoding
df = pd.get_dummies(df, columns=["JOB"], drop_first=True)

# Kiểm tra kết quả
print(df.head())

In [ ]:
print(df.filter(like="JOB_").sum())

In [ ]:
# Chuyển đổi tất cả các cột JOB_* từ boolean về integer
job_cols = df.filter(like="JOB_").columns
df[job_cols] = df[job_cols].astype(int)
# Kiểm tra lại
print(df[job_cols].head())

In [ ]:
df

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

In [ ]:
X = df.drop(columns=["BAD"])
y = df["BAD"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print(f"Train size: {len(X_train)}, Test size: {len(X_test)}")

In [ ]:
# Kiểm tra kiểu dữ liệu của các cột
print(X_train.dtypes)

In [ ]:
scaler = MinMaxScaler()

cols = X_train.columns
X_train[cols] = scaler.fit_transform(X_train[cols])
X_test[cols] = scaler.transform(X_test[cols])

In [ ]:
sm = SMOTE(random_state=42)
X_train,y_train = sm.fit_resample(X_train,y_train)
print("Dimension of X_train_sm Shape:", X_train.shape)
print("Dimension of y_train_sm Shape:", y_train.shape)

1. Logistic Regression

In [ ]:
lr = LogisticRegression(penalty='l1', C=0.9, solver='saga', n_jobs=-1)
lr.fit(X_train, y_train)

In [ ]:
def evaluation(model):
    y_test_pred = model.predict(X_test)

    print(confusion_matrix(y_test, y_test_pred))
    print(classification_report(y_test, y_test_pred))
    print("Accuracy of TEST data: {:.2f}%".format(100 * accuracy_score(y_test, y_test_pred)))
    print("F1 Score of TEST data: {:.2f}%".format(100 * f1_score(y_test, y_test_pred, average="macro")))
    print("Recall of TEST data: {:.2f}%".format(100 * recall_score(y_test, y_test_pred, average="macro")))
    print("="*50)

    # Calculate the RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    print("RMSE: ", rmse)

    # Calculate ROC AUC
    roc_auc = roc_auc_score(y_test, y_test_pred, average=None)
    print("ROC AUC score: ", roc_auc)

evaluation(lr)

2. Random Forest

In [ ]:
regr_rfr = RandomForestClassifier(random_state=42,oob_score=True)
regr_rfr.fit(X_train,y_train)

In [ ]:
evaluation(regr_rfr)

In [ ]:
!pip install optuna

In [ ]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, recall_score, classification_report, roc_auc_score
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
import numpy as np

# === Bước 1: Hàm Recall tùy chỉnh cho lớp 1 (BAD) ===
def custom_recall_score(y_true, y_pred):
    return recall_score(y_true, y_pred, pos_label=1)

recall_scorer = make_scorer(custom_recall_score)

# === Bước 2: Hàm objective cho Optuna ===
def objective(trial):
    # Tập siêu tham số cho RandomForest
    params = {
        "classifier__n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "classifier__max_depth": trial.suggest_int("max_depth", 2, 22),
        "classifier__min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "classifier__min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
        "classifier__max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
        "classifier__bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
        "classifier__ccp_alpha": trial.suggest_float("ccp_alpha", 0.0, 0.1)
    }

    # Tạo pipeline với SMOTE và RandomForestClassifier
    pipeline = ImbPipeline([
        ('smote', SMOTE(random_state=42)),
        ('classifier', RandomForestClassifier(random_state=42))
    ])
    pipeline.set_params(**params)

    # Tính điểm Recall trung bình từ 5-fold cross-validation
    recall = cross_val_score(pipeline, X_train, y_train, scoring=recall_scorer, cv=5).mean()
    return recall

# === Bước 3: Tạo và chạy Optuna study ===
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=500, timeout=600)

# === Bước 4: In kết quả tốt nhất ===
print("✅ Best Recall (train CV):", study.best_value)
print("📌 Best hyperparameters:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

# === Bước 5: Huấn luyện lại mô hình với các siêu tham số tốt nhất ===
best_params = {key.replace("classifier__", ""): value for key, value in study.best_params.items()}
best_rf_model = RandomForestClassifier(**best_params, random_state=42)

# Tạo pipeline cho huấn luyện cuối cùng
final_pipeline = ImbPipeline([
    ('smote', SMOTE(random_state=42)),
    ('classifier', best_rf_model)
])

# Huấn luyện mô hình trên toàn bộ tập train
final_pipeline.fit(X_train, y_train)

# === Bước 6: Hàm evaluation tùy chỉnh ===
def evaluation(model, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test):
    y_test_pred = model.predict(X_test)

    # Dự đoán xác suất cho ROC AUC
    y_test_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None


    # In kết quả cho tập test
    print("\n==================================================")
    print("Evaluation for Test Data")
    print("==================================================")
    print(classification_report(y_test, y_test_pred))
    print(f"Accuracy of TEST data: {100 * np.mean(y_test_pred == y_test):.2f}")
    print(f"Recall of TEST data (class 1): {100 * recall_score(y_test, y_test_pred, pos_label=1):.2f}")

    # ROC AUC score (nếu có xác suất)
    if y_test_proba is not None:
        print(f"ROC AUC score: {roc_auc_score(y_test, y_test_proba):.4f}")

    # RMSE (dựa trên xác suất, nếu có)
    if y_test_proba is not None:
        rmse = np.sqrt(np.mean((y_test - y_test_proba) ** 2))
        print(f"RMSE: {rmse:.4f}")

# === Bước 7: Đánh giá mô hình tốt nhất ===
print("\n==================================================")
print("Evaluation for the best Random Forest Model")
evaluation(final_pipeline)

3. Decision Tree

In [ ]:
tree_model = DecisionTreeClassifier()
tree_model.fit(X_train, y_train)

In [ ]:
evaluation(tree_model)

In [ ]:
import optuna
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, recall_score, classification_report, roc_auc_score
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
import numpy as np
 
# Giả định X_train, y_train, X_test, y_test đã được định nghĩa
# Nếu chưa có, bạn cần thêm đoạn mã chia dữ liệu:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
 
# === Bước 1: Hàm Recall tùy chỉnh cho lớp 1 (BAD) ===
def custom_recall_score(y_true, y_pred):
    return recall_score(y_true, y_pred, pos_label=1)
 
recall_scorer = make_scorer(custom_recall_score)
 
# === Bước 2: Hàm objective cho Optuna ===
def objective(trial):
    # Tập siêu tham số cần tìm (giới hạn max_depth để giảm overfitting)
    params = {
        "classifier__criterion": trial.suggest_categorical("criterion", ["gini", "entropy"]),
        "classifier__max_depth": trial.suggest_int("max_depth", 2, 22),  # Giảm từ 40 xuống 20
        "classifier__min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "classifier__min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
        "classifier__max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
        "classifier__ccp_alpha": trial.suggest_float("ccp_alpha", 0.0, 0.1)  # Thêm ccp_alpha
    }
 
    # Tạo pipeline với SMOTE và DecisionTreeClassifier
    pipeline = ImbPipeline([
        ('smote', SMOTE(random_state=42)),
        ('classifier', DecisionTreeClassifier(random_state=42))
    ])
    pipeline.set_params(**params)
 
    # Tính điểm Recall trung bình từ 5-fold cross-validation
    recall = cross_val_score(pipeline, X_train, y_train, scoring=recall_scorer, cv=5).mean()
    return recall
 
# === Bước 3: Tạo và chạy Optuna study ===
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=500, timeout=600)  # Giảm n_trials từ 20000 xuống 100
 
# === Bước 4: In kết quả tốt nhất ===
print("✅ Best Recall (train CV):", study.best_value)
print("📌 Best hyperparameters:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")
 
# === Bước 5: Huấn luyện lại mô hình với các siêu tham số tốt nhất ===
best_params = {key.replace("classifier__", ""): value for key, value in study.best_params.items()}
best_tree_model = DecisionTreeClassifier(**best_params, random_state=42)
 
# Tạo pipeline cho huấn luyện cuối cùng
final_pipeline = ImbPipeline([
    ('smote', SMOTE(random_state=42)),
    ('classifier', best_tree_model)
])
 
# Huấn luyện mô hình trên toàn bộ tập train
final_pipeline.fit(X_train, y_train)
 
# === Bước 6: Hàm evaluation tùy chỉnh ===
def evaluation(model, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test):
    # Dự đoán trên tập train và test
    y_test_pred = model.predict(X_test)
 
    # Dự đoán xác suất cho ROC AUC
    y_test_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

 
    # In kết quả cho tập test
    print("\n==================================================")
    print("Evaluation for Test Data")
    print("==================================================")
    print(classification_report(y_test, y_test_pred))
    print(f"Accuracy of TEST data: {100 * np.mean(y_test_pred == y_test):.2f}")
    print(f"Recall of TEST data (class 1): {100 * recall_score(y_test, y_test_pred, pos_label=1):.2f}")
 
    # ROC AUC score (nếu có xác suất)
    if y_test_proba is not None:
        print(f"ROC AUC score: {roc_auc_score(y_test, y_test_proba):.4f}")
 
    # RMSE (dựa trên xác suất, nếu có)
    if y_test_proba is not None:
        rmse = np.sqrt(np.mean((y_test - y_test_proba) ** 2))
        print(f"RMSE: {rmse:.4f}")
 
# === Bước 7: Đánh giá mô hình tốt nhất ===
print("\n==================================================")
print("Evaluation for the best Decision Tree Model")
evaluation(final_pipeline)

4. CatBoost 

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
 

cat_model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    eval_metric='AUC',
    random_seed=42
)
 
cat_model.fit(X_train, y_train, early_stopping_rounds=50)
 



In [ ]:
def evaluation(model):
    y_test_pred = model.predict(X_test)

    # Nếu đầu ra là xác suất (continuous), cần chuyển sang nhãn (binary)
    if y_test_pred.ndim == 2 and y_test_pred.shape[1] == 1:
        y_test_pred = y_test_pred.ravel()  # flatten if shape is (n_samples, 1)

    y_test_pred_class = (y_test_pred > 0.5).astype(int)

    print(classification_report(y_test, y_test_pred_class))
    print("Accuracy of TEST data: {:.2f}%".format(100 * accuracy_score(y_test, y_test_pred_class)))
    print("F1 Score of TEST data: {:.2f}%".format(100 * f1_score(y_test, y_test_pred_class, average="macro")))
    print("Recall of TEST data: {:.2f}%".format(100 * recall_score(y_test, y_test_pred_class, average="macro")))
    print("="*50)

    # Calculate the RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    print("RMSE: ", rmse)

    # Calculate ROC AUC
    roc_auc = roc_auc_score(y_test, y_test_pred)
    print("ROC AUC score: ", roc_auc)

evaluation(cat_model)

5. LightGBM

In [ ]:
!pip install lightgbm

In [ ]:
from lightgbm import LGBMClassifier

# Khởi tạo và huấn luyện mô hình LightGBM
light_model = LGBMClassifier(random_state=42)
light_model.fit(X_train, y_train)

In [ ]:
def evaluation(model):
    y_test_pred = model.predict(X_test)

    # Nếu đầu ra là xác suất (continuous), cần chuyển sang nhãn (binary)
    if y_test_pred.ndim == 2 and y_test_pred.shape[1] == 1:
        y_test_pred = y_test_pred.ravel()  # flatten if shape is (n_samples, 1)

    y_test_pred_class = (y_test_pred > 0.5).astype(int)

    print(classification_report(y_test, y_test_pred_class))
    print("Accuracy of TEST data: {:.2f}%".format(100 * accuracy_score(y_test, y_test_pred_class)))
    print("F1 Score of TEST data: {:.2f}%".format(100 * f1_score(y_test, y_test_pred_class, average="macro")))
    print("Recall of TEST data: {:.2f}%".format(100 * recall_score(y_test, y_test_pred_class, average="macro")))
    print("="*50)

    # Calculate the RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    print("RMSE: ", rmse)

    # Calculate ROC AUC
    roc_auc = roc_auc_score(y_test, y_test_pred)
    print("ROC AUC score: ", roc_auc)

evaluation(light_model)

6. XGBoost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
x_model = XGBClassifier(use_label_encoder=False,eval_metric='logloss',random_state=42)
x_model.fit(X_train, y_train)

In [ ]:
def evaluation(model):
    y_test_pred = model.predict(X_test)

    # Nếu đầu ra là xác suất (continuous), cần chuyển sang nhãn (binary)
    if y_test_pred.ndim == 2 and y_test_pred.shape[1] == 1:
        y_test_pred = y_test_pred.ravel()  # flatten if shape is (n_samples, 1)

    y_test_pred_class = (y_test_pred > 0.5).astype(int)

    print(classification_report(y_test, y_test_pred_class))
    print("Accuracy of TEST data: {:.2f}%".format(100 * accuracy_score(y_test, y_test_pred_class)))
    print("F1 Score of TEST data: {:.2f}%".format(100 * f1_score(y_test, y_test_pred_class, average="macro")))
    print("Recall of TEST data: {:.2f}%".format(100 * recall_score(y_test, y_test_pred_class, average="macro")))
    print("="*50)

    # Calculate the RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    print("RMSE: ", rmse)

    # Calculate ROC AUC
    roc_auc = roc_auc_score(y_test, y_test_pred)
    print("ROC AUC score: ", roc_auc)

evaluation(x_model)

7. SVM

In [ ]:
# SMOTETomek
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# Khởi tạo các kỹ thuật resampling
sampling_strategy = {0: 2384, 1:2384}
over = SMOTE(sampling_strategy='auto', random_state=42)
under = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
pipeline = Pipeline(steps=[('over', over), ('under', under)])

# Áp dụng kỹ thuật resampling vào tập huấn luyện
X_train_resampled, y_train_resampled = pipeline.fit_resample(X_train, y_train)

new_train_data = pd.concat([X_train_resampled, y_train_resampled], axis=1)
print(new_train_data[['BAD']].value_counts(normalize=True))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, roc_curve

# Chuẩn hóa dữ liệu (Đưa về mean = 0, variance = 1)
scaler = StandardScaler()
X_train_resampled_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)  # x <--   (x - mean(x_train))/std(x_train)  (Đảm bảo tập test được chuẩn hóa như tập train.)

# Huấn luyện SVM với gamma = 1
svm_rbf = SVC(kernel='rbf', C=2, gamma=1, class_weight='balanced',probability=True, random_state=42)
svm_rbf.fit(X_train_resampled_scaled, y_train_resampled)

# Đánh giá trên tập train (sau resampling)
y_train_pred = svm_rbf.predict(X_train_resampled_scaled)
print("The evaluation of training set (using SMOTETomek):")
print(classification_report(y_train_resampled, y_train_pred))

# Đánh giá trên tập test
y_test_pred = svm_rbf.predict(X_test_scaled)
print("\nThe evaluation of test set:")
print(classification_report(y_test, y_test_pred))

auc_train = roc_auc_score(y_train_resampled, y_train_pred)
auc_test = roc_auc_score(y_test, y_test_pred)

print(f"AUC score trên tập train: {auc_train:.4f}")
print(f"AUC score trên tập test: {auc_test:.4f}")

8. Compare these model

In [ ]:
models = {
    "Logistic Regression": LogisticRegression(penalty='l1', C=0.9, solver='saga', n_jobs=-1),
    "Random Forest": RandomForestClassifier(n_estimators = 266,max_depth = 9, min_samples_split = 2, min_samples_leaf = 4,max_features='sqrt', bootstrap = False, ccp_alpha = 0.0002841158182021458),
    "XGBoost": XGBClassifier(use_label_encoder=False,eval_metric='logloss',random_state=42),
    "Decision Tree": DecisionTreeClassifier(criterion = 'gini', max_depth = 21, min_samples_split = 6, min_samples_leaf = 1, max_features = None, ccp_alpha = 0.00015159440001651534),
    "CatBoost": CatBoostClassifier(iterations=500,learning_rate=0.05,depth=6,eval_metric='AUC',random_seed=42),
    "SVM": SVC(kernel='rbf', C=2, gamma=1, class_weight='balanced',probability=True, random_state=42),
    "LightGBM": LGBMClassifier(random_state=42)
}

In [ ]:
def evaluate_model(name,model,X_train,y_train,X_test,y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:,1]
    return {
        "Model": name,
        "Accuracy": accuracy_score(y_test,y_pred),
        "Precision": precision_score(y_test,y_pred),
        "Recall": recall_score(y_test,y_pred),
        "F1 Score": f1_score(y_test,y_pred),
        "AUC": roc_auc_score(y_test,y_proba)
    }

In [ ]:
results = []
for name, model in models.items():
    print(f"Training: {name}")
    res = evaluate_model(name,model,X_train,y_train,X_test,y_test)
    results.append(res)

In [ ]:
results_df = pd.DataFrame(results).sort_values(by="AUC",ascending=False)
results_df

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(x="Accuracy", y="Model", data=results_df, palette="coolwarm")
plt.title("Model Performance (AUC)")
plt.tight_layout()
plt.show()

In [ ]:
best_model_name = results_df.iloc[0]['Model']
best_model_name

In [ ]:
# === 11. Save best model and preprocessor ===
import joblib
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

# Get the best model based on AUC
best_model_name = results_df.iloc[0]['Model']
best_model = models[best_model_name]

# Re-fit the best model (to ensure it's trained with all data)
best_model.fit(X_train, y_train)

# Save the best model
joblib.dump(best_model, 'best_model.pkl')

# Save the scaler
joblib.dump(scaler, 'scaler.pkl')

# === 12. Prediction function for new samples ===
def predict_sample(new_data):
    """
    Predict using the saved model and scaler.
    
    Args:
        new_data (DataFrame): New data with same structure as training data
    
    Returns:
        tuple: (prediction, probability)
    """
    # Load model and scaler
    model = joblib.load('best_model.pkl')
    scaler = joblib.load('scaler.pkl')
    
    # Preprocess the new data
    # 1. Handle categoricals (JOB_* columns should already exist if using get_dummies)
    # 2. Ensure column order matches training
    new_data = new_data[X_train.columns]  # Reorder columns
    
    # Scale numerical features
    new_data_scaled = scaler.transform(new_data)
    
    # Predict
    prediction = model.predict(new_data_scaled)
    probability = model.predict_proba(new_data_scaled)[:, 1]
    
    return prediction, probability

# === 13. Example usage ===
# Create sample data (with all features including dummy columns)
sample_data = pd.DataFrame({
    'LOAN': [10000],
    'MORTDUE': [20000],
    'VALUE': [25000],
    'DEROG': [0],
    'DELINQ': [0],
    'CLAGE': [100],
    'NINQ': [1],
    'CLNO': [5],
    'DEBTINC': [35],
    'YOJ': [5],
    'JOB_Office': [0],
    'JOB_Other': [1],
    'JOB_ProfExe': [0],
    'JOB_Sales': [0],
    'JOB_Self': [0]
}, index=[0])

# Predict
prediction, probability = predict_sample(sample_data)
print(f"\nPrediction: {'BAD' if prediction[0] == 1 else 'GOOD'}")
print(f"Probability: {probability[0]:.2f}")

# === 14. Detailed evaluation of best model ===
print("\n=== Best Model Evaluation ===")
print(f"Model: {best_model_name}")

# Predict on test set
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

# Print metrics
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nAUC-ROC:", roc_auc_score(y_test, y_proba))
